In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC

In [2]:
import pickle
with open('dataframe.pkl', 'rb') as file:
    df_loaded = pickle.load(file)

In [4]:
df=df_loaded
df

,radius_mean,texture_mean,perimeter_mean,area_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,fractal_dimension_mean,radius_se,...,area_worst,compactness_worst,concavity_worst,symmetry_worst,fractal_dimension_worst,smoothness_mean,texture_worst,smoothness_worst,concave_points_worst,diagnosis
id,,,,,,,,,,,,,,,,,,,,,
842302.0,0.521037,0.033070,0.545989,0.363733,0.792037,0.730526,0.768949,0.686364,0.196541,0.356147,...,0.450698,0.616677,0.568610,0.721311,0.418864,0.839315,0.185276,0.759737,0.962990,1
842517.0,0.643144,0.397828,0.615783,0.501591,0.181768,0.211538,0.366806,0.379798,0.186047,0.156437,...,0.435214,0.148757,0.192971,0.281540,0.222878,0.333834,0.397418,0.399343,0.674891,1
84300903.0,0.601496,0.569595,0.595743,0.449417,0.431017,0.480526,0.668583,0.509596,0.384019,0.229622,...,0.374508,0.381154,0.359744,0.486576,0.213433,0.707163,0.471389,0.592679,0.881713,1
84358402.0,0.629893,0.228529,0.630986,0.489290,0.347893,0.481986,0.545217,0.378283,0.314848,0.233822,...,0.341575,0.166732,0.319489,0.189831,0.142595,0.567503,0.162247,0.526983,0.589623,1
843786.0,0.258839,0.295656,0.267984,0.141506,0.461996,0.384129,0.422844,0.518687,0.196541,0.080753,...,0.136748,0.479232,0.427716,0.574958,0.454939,0.980478,0.409281,0.918348,0.631713,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
926125.0,0.659709,0.759131,0.685578,0.510498,0.626403,0.772639,0.770518,0.550000,0.908766,0.308057,...,0.401543,0.375391,0.527077,0.324067,0.286567,0.711668,0.606769,0.557954,0.922351,1
926424.0,0.690000,0.625864,0.678668,0.566490,0.296055,0.593720,0.726085,0.336364,0.159809,0.385479,...,0.452664,0.172886,0.328035,0.117605,0.105667,0.728187,0.501745,0.560770,0.804064,1
926682.0,0.622320,0.915104,0.604036,0.474019,0.257714,0.350536,0.511814,0.349495,0.106142,0.236828,...,0.379915,0.154228,0.256789,0.239249,0.074315,0.529959,0.915213,0.331769,0.590711,1


In [5]:
y=df['diagnosis']
X = df.drop(columns=['diagnosis'])

In [9]:
# Fit models and determine if a feature is selected (1) or not (0)
lasso = Lasso(alpha=0.01).fit(X, y)
lasso_selected = (np.abs(lasso.coef_) > 0).astype(int)

svm = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, y)
svm_selected = (np.abs(svm.coef_[0]) > 0).astype(int)

gb = GradientBoostingClassifier().fit(X, y)
gb_selected = (gb.feature_importances_ > 0).astype(int)

rf = RandomForestClassifier().fit(X, y)
rf_selected = (rf.feature_importances_ > 0).astype(int)

# Create a DataFrame to store results
selection_df = pd.DataFrame({
    'Feature': X.columns,
    'Lasso': lasso_selected,
    'SVM': svm_selected,
    'GradientBoost': gb_selected,
    'RandomForest': rf_selected
})

# Sum the number of selections for each feature
selection_df['Sum'] = selection_df[['Lasso', 'SVM', 'GradientBoost', 'RandomForest']].sum(axis=1)

# Output the results
print(selection_df)

                    Feature  Lasso  SVM  GradientBoost  RandomForest  Sum
0               radius_mean      0    0              1             1    2
1              texture_mean      0    0              1             1    2
2            perimeter_mean      0    0              1             1    2
3                 area_mean      0    0              1             1    2
4          compactness_mean      0    0              1             1    2
5            concavity_mean      0    0              1             1    2
6       concave_points_mean      1    0              1             1    3
7             symmetry_mean      0    0              1             1    2
8    fractal_dimension_mean      0    0              1             1    2
9                 radius_se      0    0              1             1    2
10               texture_se      0    0              1             1    2
11             perimeter_se      0    0              1             1    2
12                  area_se      0    

In [12]:
 #Selecting variables with a sum of selections >= 3
final_var = selection_df[selection_df['Sum'] >= 3]['Feature'].tolist()
df_model = df[final_var].copy()
df_model['diagnosis'] = df['diagnosis'].copy()

# Output the result to verify
df_model.info()

<class 'pandas.core.frame.DataFrame'>
Index: 547 entries, 842302.0 to 927241.0
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   concave_points_mean   547 non-null    float64
 1   radius_worst          547 non-null    float64
 2   texture_worst         547 non-null    float64
 3   concave_points_worst  547 non-null    float64
 4   diagnosis             547 non-null    int32  
dtypes: float64(4), int32(1)
memory usage: 23.5 KB
